In [ ]:
#### The cutting stock object is the environment
# %pip install -i https://pypi.gurobi.com gurobipy
from gurobipy import GRB
import gurobipy as gp
import numpy as np
import pandas as pd
import random
from copy import deepcopy
import numpy as np
import gurobipy as gp
import math
import numpy as np
from copy import deepcopy
from collections import namedtuple
from typing import List
from tqdm import tqdm
import random 

Looking in indexes: https://pypi.gurobi.com


In [1]:



#### define transition object to hold the transition data
class Transition(object):
# s0_augmented, a0, r, is_done, s1_augmented, total

    def __init__(self, s0_augmented, a0, r:float, is_done:bool, s1_augmented,action_info_0,total_0, total_1):
          self.data = (s0_augmented, a0, r, is_done, s1_augmented, action_info_0, total_0, total_1)

    @property
    def s0(self): 
          return self.data[0]

    @property
    def a0(self): 
        return self.data[1]

    @property
    def reward(self): 
        return self.data[2]
    
    @property
    def is_done(self): 
        return self.data[3]

    @property
    def s1(self): 
        return self.data[4]
    
    @property
    def action_info_0(self): 
        return self.data[5]



    @property
    def total_0(self): 
        return self.data[6]

    @property
    def total_1(self): 
        return self.data[7]

    def __iter__(self):
        return iter(self.data)
    
    def __str__(self):
        edge_num1 = len(self.data[0][1][1])
        edge_num2 = len(self.data[-1][1][1])
        result = 'transit from bipartite graph with edge' + str(edge_num1) + ' to ' + str(edge_num2) +' collecting the reward ' + str(self.data[2])
            # return 'transit from bipartite graph ' + col_string1 + ' connects to ' + con_string1 + ' to ' + col_string2 + ' connects to ' + con_string2 +' collecting the reward ' + str(self.data[2])
        return result
        

    
############
def show_final_solution(cut):
    use = [math.ceil(i) for i in cut.ColumnSol_Val]
    for i, p in enumerate(cut.current_patterns):
        if use[i]>0:
            print('Pattern ', i, ': how often we should cut: ', use[i])
            print('----------------------')
            for j,order in enumerate(p):
                if order >0:
                    print('order ', j, ' how much: ', order)
            print()

    print('Total number of rolls used: ', int(np.asarray(cut.ColumnSol_Val).sum()))
##############


class Episode(object):
  def __init__(self, e_id:int = 0) -> None:
      self.total_reward = 0  
      self.trans_list = []    
      self.name = str(e_id)   

  def push(self, trans:Transition) -> float:
      self.trans_list.append(trans)
      self.total_reward += trans.reward 
      return self.total_reward

  @property
  def len(self):
      return len(self.trans_list)

  def __str__(self):
      return "episode {0:<4} {1:>4} steps,total reward:{2:<8.2f}".\
          format(self.name, self.len, self.total_reward)

  def print_detail(self):
      print("detail of ({0}):".format(self))
      for i,trans in enumerate(self.trans_list):
          print("step{0:<4} ".format(i),end=" ")
          print(trans)

  def pop(self) -> Transition:
      '''normally this method shouldn't be invoked.
      '''
      if self.len > 1:
          trans = self.trans_list.pop()
          self.total_reward -= trans.reward
          return trans
      else:
          return None

  def is_complete(self) -> bool:
      '''check if an episode is an complete episode
      '''
      if self.len == 0: 
          return False 
      return self.trans_list[self.len-1].is_done

  def sample(self, batch_size = 1):   
      '''随即产生一个trans
      '''
      return random.sample(self.trans_list, k = batch_size)

  def __len__(self) -> int:
      return self.len


class Experience(object):
  '''this class is used to record the whole experience of an agent organized
  by an episode list. agent can randomly sample transitions or episodes from
  its experience.
  '''
  def __init__(self, capacity:int = 20000):
      self.capacity = capacity    
      self.episodes = []         
      self.next_id = 0            
      self.total_trans = 0        
      
  def __str__(self):
      return "exp info:{0:5} episodes, memory usage {1}/{2}".\
              format(self.len, self.total_trans, self.capacity)

  def __len__(self):
      return self.len

  @property
  def len(self):
      return len(self.episodes)

  def _remove(self, index = 0):      
      '''
          remove an episode, defautly the first one.
          args: 
              the index of the episode to remove
          return:
              if exists return the episode else return None
      '''
      if index > self.len - 1:
          raise(Exception("invalid index"))
      if self.len > 0:
          episode = self.episodes[index]
          self.episodes.remove(episode)
          self.total_trans -= episode.len
          return episode
      else:
          return None

  def _remove_first(self):
      self._remove(index = 0)

  def push(self, trans): 

      if self.capacity <= 0:
          return
      while self.total_trans >= self.capacity: 
          episode = self._remove_first()
      cur_episode = None
      if self.len == 0 or self.episodes[self.len-1].is_complete():
          cur_episode = Episode(self.next_id)
          self.next_id += 1
          self.episodes.append(cur_episode)
      else:
          cur_episode = self.episodes[self.len-1]
      self.total_trans += 1
      return cur_episode.push(trans)      #return  total reward of an episode

  def sample(self, batch_size=32): # sample transition
      '''randomly sample some transitions from agent's experience.abs
      args:
          number of transitions need to be sampled
      return:
          list of Transition.
      '''
      # sample_trans = []
      sample_trans = np.asarray([])
      for _ in range(batch_size):
          index = int(random.random() * self.len)
          sample_trans = np.append(sample_trans,self.episodes[index].sample())

          # sample_trans += self.episodes[index].sample()
      return sample_trans

  def sample_episode(self, episode_num = 1):  # sample episode
      '''
      '''
      return random.sample(self.episodes, k = episode_num)

  @property
  def last_episode(self):
      if self.len > 0:
          return self.episodes[self.len-1]
      return None

  

  
          




In [ ]:
import numpy as np
from sys import exit
import os


def readInstanceN():
    # Take in input instance name
    print("Instance name: [r|c|rc][NNN]")
    INSTANCE_NAME = None
    try:
        INSTANCE_NAME = input()
    except Exception as e:
        print("Invalid instance. Exit."); exit(1)
    if INSTANCE_NAME.endswith(".txt"):
        INSTANCE_NAME = INSTANCE_NAME[:-4]
    INSTANCE_FILENAME = os.path.join("solomon-instances", INSTANCE_NAME+".txt")
    if not INSTANCE_NAME or not os.path.exists(INSTANCE_FILENAME):
        print("Instance does not exists. Exit."); exit(1)

    # Take in input customers number
    print("Select number of costumers (1-100):")
    n = None
    try:
        n = int(input())
    except Exception as e:
        print("Invalid number of costumers. Exit."); exit(1)
    if not n or n < 1 or n > 100:
        print("Invalid number of costumers. Exit."); exit(1)
    return INSTANCE_NAME, INSTANCE_FILENAME, n


def readData(filename, n):
    stream = ""
    with open(filename, "r") as file:
        stream = file.readlines()
    if stream == "":
        print("Error in reading file")
    else:
        print("Read file", filename)

    vehicleNumber, capacity = [int(i) for i in stream[4].split()]
    fields = ("CUST-NO.", "XCOORD.", "YCOORD.", "DEMAND", "READY-TIME", \
                "DUE-DATE", "SERVICE-TIME")
    data = list()
    for i in range(9, len(stream)):
        if stream[i] == "\n":
            continue
        val = stream[i].split()
        if len(val) != len(fields):
            print("Error in reading data")
            continue
        customer = dict(zip(fields, val))
        data.append(customer)

    # Consider only depot + 50 customers
    data = data[0:n+1]
    data.append(data[0]) # The depot is represented by two identical
                         # nodes: 0 and n+1
    data[-1]["CUST-NO."] = "51"
    x = []; y = []; q = []; a = []; b = []
    for customer in data:
        x.append(int(customer["XCOORD."]))
        y.append(int(customer["YCOORD."]))
        q.append(int(customer["DEMAND"]))
        a.append(int(customer["READY-TIME"]))
        b.append(int(customer["DUE-DATE"]))

    return vehicleNumber, capacity, x, y, q, a, b


def createDistanceMatrix(x, y):
    n = len(x)
    d = np.zeros((n,n))
    for i in range(n):
        for j in range(i+1,n):
            p1 = np.array([x[i], y[i]])
            p2 = np.array([x[j], y[j]])
            d[i,j] = d[j,i] = int(round(np.linalg.norm(p1-p2)))
    return d


def addRoutesToMaster(routes, mat, costs, d):
    for i in range(len(routes)):
        cost = d[routes[i][0],routes[i][1]]
        for j in range(1,len(routes[i])-1):
            cost += d[routes[i][j], routes[i][j+1]]
            mat[routes[i][j]-1,i] += 1
        costs[i] = cost


In [ ]:
b1 = b2 = b3 = bs = be = br = 1/3


def insertNode(route, node, position, s, arr, d, a):
    newRoute = route[:]
    newRoute.insert(position, node)
    newS = []; newArr = []
    for i in range(position):
        newS.append(s[i])
        newArr.append(arr[i])
    for i in range(position, len(newRoute)):
        newArr.append(newS[i-1] + d[newRoute[i-1], newRoute[i]])
        newS.append(max(newArr[i], a[i]))
    return newRoute, newS, newArr


def routeIsFeasible(route, a, b, s, d, q, Q):
    cap = sum([q[node] for node in route])
    if cap > Q:
        return False
    for i in range(len(route)):
        if not ((s[i] >= a[route[i]]) and (s[i] <= b[route[i]])):
            return False
    return True


def computeISIULD(posU, route, arr, s, a, b, d, Jminu):
    u = route[posU]
    posI = posU-1; posJ = posU+1
    i = route[posI]; j = route[posJ]
    IS = arr[posU] - a[u]
    IU = 1/(max([len(Jminu), 1])) * sum([max([b[n]-a[u]-d[u,n], b[u]-a[n]-d[u,n]]) \
                                for n in Jminu])
    c1 = (d[i,u] + d[u,j] - d[i,j])
    c2 = ((b[j]- (arr[posI] + d[i,j])) - \
                (b[j] - (arr[posU] + d[i,j])))
    c3 = (b[u] - (arr[posI] + d[i,u]))
    LD = b1*c1 + b2*c2 + b3*c3

    return IS, IU, LD


def computeImpact(IS, IU, LD, feasiblePositions):
    IR = sum(LD)/len(feasiblePositions)
    bestImpact = None
    bestPosition = None
    for i in range(len(feasiblePositions)):
        impact = IS[i] + IU[i] + IR
        if bestPosition == None or impact < bestImpact:
            bestImpact = impact
            bestPosition = feasiblePositions[i]

    return bestPosition, bestImpact


def computeRouteCost(route, d):
    cost = 0
    for i in range(len(route)-1):
        cost += d[route[i], route[i+1]]
    return cost


def initializePathsWithImpact(d, n, a, b, q, Q):
    J = list(range(1,n+1))
    routes = []
    costs = []

    while J:
        # Find furthest node from depot in J and initialize route with it
        far = -1
        max_dist = -1
        for j in J:
            if d[0,j] > max_dist:
                far = j
                max_dist = d[0,j]

        route = [0, far, n+1]
        arr = [0, d[0,far]]
        s = [0, max([a[far], arr[1]])]
        arr.append(s[1] + d[far,n+1])
        s.append(max(arr[2], a[n+1]))
        J.remove(far)

        feasible = J[:]

        while feasible:
            proposals = dict()
            for u in feasible:
                bestImpact = None
                bestPosition = None
                Jminu = J[:]
                Jminu.remove(u)
                feasiblePositions = []
                IS = IU = LD = []
                for pos in range(1, len(route)):
                    newRoute, newS, newArr = insertNode(route, u, pos, s, \
                                                        arr, d, a)
                    if routeIsFeasible(newRoute, a, b, newS, d, q, Q):
                        feasiblePositions.append(pos)
                        Is, Iu, Ld = computeISIULD(pos, newRoute, newArr, \
                                                    newS, a, b, d, Jminu)
                        IS.append(Is); IU.append(Iu); LD.append(Ld)

                if not feasiblePositions:
                    feasible.remove(u)
                else:
                    bestPosition, bestImpact = computeImpact(IS, IU, LD, \
                                                             feasiblePositions)
                    proposals[bestImpact] = (u, bestPosition)
                # END FOR
            # prendo miglior impact
            if proposals:
                nodeToInsert, insertPos = proposals[min(list(proposals.keys()))]
                # aggiungo nodo in posizione
                route, s, arr = insertNode(route, nodeToInsert, insertPos, s, \
                                            arr, d, a)
                # rimuovo nodo da J e da feasible
                feasible.remove(nodeToInsert)
                J.remove(nodeToInsert)
            # END WHILE

        routes.append(route)
        costs.append(computeRouteCost(route, d))
        # END WHILE
    # print("Impact routes:\n", routes)
    return routes


In [ ]:
import numpy as np
import gurobipy as gp
import math


def createMasterProblem(A, costs, n, vehicleNumber):
    model = gp.Model("Master problem")
    model.Params.OutputFlag = 0
    y = model.addMVar(shape=A.shape[1], vtype=gp.GRB.CONTINUOUS, name="y")
    model.setObjective(costs @ y, gp.GRB.MINIMIZE)
    # Constraints
    model.addConstr(A @ y == np.ones(A.shape[0]))
    model.write("MasterModel.lp")

    return model


def reduceTimeWindows(n, d, readyt, duedate):
    a = readyt[:]; b = duedate[:]
    update = True
    while update:
        update = False
        for k in range(1,n+1):
            # Ready Time
            minArrPred = min([b[k], \
                            min([a[i] + d[i,k] for i in range(n+1) if i!=k])])
            minArrNext = min([b[k], \
                            min([a[j] - d[k,j] for j in range(1,n+2) if j!=k])])
            newa = int(max([a[k], minArrPred, minArrNext]))
            if newa != a[k]:
                update = True
            a[k] = newa

            # Due date
            maxDepPred = max([a[k],
                            max([b[i] + d[i,k] for i in range(n+1) if i!=k])])
            maxDepNext = max([a[k],
                            max([b[j] - d[k,j] for j in range(1,n+2) if j!=k])])
            newb = int(min([b[k], maxDepPred, maxDepNext]))
            if newb != b[k]:
                update = True
            b[k] = newb
    return a,b


def subProblem(n, q, d, readyt, duedate, rc, Q):
    
    M = gp.GRB.INFINITY     # 1e+100
    # Time windows reduction
    a,b = reduceTimeWindows(n, d, readyt, duedate)
    # Reduce max capacity to boost algorithm
    if sum(q) < Q:
        Q = sum(q)
    T = max(b)

    # Init necessary data structure
    f = list()  # paths cost data struct
    p = list()  # paths predecessor data struct
    f_tk = list()     # cost of the best path that does not pass for
                      # predecessor (we'll call it alternative path)
    paths = []
    paths_tk = []
    for j in range(n+2):
        paths.append([])
        paths_tk.append([])
        for qt in range(Q-q[j]):
            paths[-1].append([])
            paths_tk[-1].append([])
            for tm in range(b[j]-a[j]):
                paths[-1][-1].append([])
                paths_tk[-1][-1].append([])
        mat = np.zeros((Q-q[j], b[j] - a[j]))
        p.append(mat - 1)
        f.append(mat + M)
        f_tk.append(mat + M)
    f[0][0,0] = 0
    f_tk[0][0,0] = 0
    L = set()   # Node to explore
    L.add(0)

    # Algorithm



    while L:
        

        i = L.pop()
        if i == n+1:
            continue

        # Explore all possible arcs (i,j)
        for j in range(1,n+2):
            if i == j:
                continue
            for q_tk in range(q[i], Q-q[j]):
                for t_tk in range(a[i], b[i]):
                    if p[i][q_tk-q[i], t_tk-a[i]] != j:
                        if f[i][q_tk-q[i], t_tk-a[i]] < M:
                            for t in range(max([a[j], int(t_tk+d[i,j])]),\
                                                b[j]):
                                if f[j][q_tk, t-a[j]]> \
                                   f[i][q_tk-q[i],t_tk-a[i]]+rc[i,j]:
                                    # if the current best path is suitable to
                                    # become the alternative path
                                    if p[j][q_tk, t-a[j]] != i \
                                       and p[j][q_tk, t-a[j]] != -1 \
                                       and f[j][q_tk, t-a[j]] < M \
                                       and f[j][q_tk,t-a[j]]<f_tk[j][q_tk,t-a[j]]:
                                        f_tk[j][q_tk,t-a[j]] = f[j][q_tk,t-a[j]]
                                        paths_tk[j][q_tk][t-a[j]] = \
                                                paths[j][q_tk][t-a[j]][:]
                                    # update f
                                    f[j][q_tk,t-a[j]] = \
                                            f[i][q_tk-q[i],t_tk-a[i]] + rc[i,j]
                                    # update path that leads to node j
                                    paths[j][q_tk][t-a[j]] = \
                                            paths[i][q_tk-q[i]][t_tk-a[i]] + [j]
                                    # Update predecessor
                                    p[j][q_tk, t-a[j]] = i
                                    L.add(j)
                                # if the path is suitable to be the alternative
                                elif p[j][q_tk, t-a[j]] != i \
                                    and p[j][q_tk, t-a[j]] != -1 \
                                    and f_tk[j][q_tk, t-a[j]] > \
                                            f[i][q_tk-q[i],t_tk-a[i]]+rc[i,j]:
                                    f_tk[j][q_tk,t-a[j]] = \
                                            f[i][q_tk-q[i],t_tk-a[i]]+rc[i,j]
                                    paths_tk[j][q_tk][t-a[j]] = \
                                            paths[i][q_tk-q[i]][t_tk-a[i]]+[j]
                    else:       # if predecessor of i is j
                        if f_tk[i][q_tk-q[i], t_tk-a[i]] < M:
                            for t in range(max([a[j],int(t_tk+d[i,j])]), \
                                                b[j]):
                                if f[j][q_tk,t-a[j]] > \
                                        f_tk[i][q_tk-q[i],t_tk-a[i]]+rc[i,j]:
                                    # if the current best path is suitable to
                                    # become the alternative path
                                    if p[j][q_tk, t-a[j]] != i \
                                        and p[j][q_tk, t-a[j]] != -1 \
                                        and f[j][q_tk, t-a[j]] < M \
                                        and f[j][q_tk,t-a[j]] < \
                                                f_tk[j][q_tk,t-a[j]]:
                                        f_tk[j][q_tk,t-a[j]] = f[j][q_tk,t-a[j]]
                                        paths_tk[j][q_tk][t-a[j]] = \
                                                paths[j][q_tk][t-a[j]][:]
                                    # update f, path and bucket
                                    f[j][q_tk,t-a[j]] = \
                                        f_tk[i][q_tk-q[i],t_tk-a[i]] + rc[i,j]
                                    paths[j][q_tk][t-a[j]] = \
                                        paths_tk[i][q_tk-q[i]][t_tk-a[i]] + [j]
                                    p[j][q_tk,t-a[j]] = i
                                    L.add(j)
                                # if the alternative path of i is suitable to
                                # be the alternate of j
                                elif p[j][q_tk, t-a[j]] != i \
                                     and p[j][q_tk, t-a[j]] != -1 \
                                     and f_tk[j][q_tk,t-a[j]] > \
                                            f_tk[i][q_tk-q[i],t_tk-a[i]]+rc[i,j]:
                                    f_tk[j][q_tk, t-a[j]] = \
                                        f_tk[i][q_tk-q[i],t_tk-a[i]]+rc[i,j]
                                    paths_tk[j][q_tk][t-a[j]] = \
                                        paths_tk[i][q_tk-q[i]][t_tk-a[i]] + [j]

    # Return all the routes with negative cost
    routes = list()
    rcosts = list()


   

    qBest, tBest = np.where(f[n+1] < -1e-9)


    for i in range(len(qBest)):
        newRoute = [0] + paths[n+1][qBest[i]][tBest[i]]
        if not newRoute in routes:
            routes.append(newRoute)
            rcosts.append(f[n+1][qBest[i]][tBest[i]])
    
    print("total iteration is", counter)
    print("total len of routes generated is", len(routes))
    return routes

In [ ]:

from collections import Counter
import os
from sys import exit


def coverCostHeuristic(bestIndex, allRoutes, allCosts, allCoverCost):
    nodes = set(range(1,n+1))
    routes = allRoutes[:]
    costs = allCosts[:]
    coverCost = allCoverCost[:]

    for node in routes[bestIndex][1:-1]:
        nodes.remove(node)
    sol = [routes[bestIndex]]
    solCost = costs[bestIndex]

    # Start loop
    while nodes:
        filteredRoutes = []
        filteredCosts = []
        filteredCoverCost = []
        for i in range(len(routes)):
            feasible = True
            for node in routes[i][1:-1]:
                if not node in nodes:
                    feasible = False
                    break
            if feasible:
                filteredRoutes.append(routes[i])
                filteredCosts.append(costs[i])
                filteredCoverCost.append(coverCost[i])
        if not filteredRoutes:
            return None

        bestIdx = filteredCoverCost.index(max(filteredCoverCost))
        sol.append(filteredRoutes[bestIdx])
        solCost += filteredCosts[bestIdx]
        for node in filteredRoutes[bestIdx][1:-1]:
            nodes.remove(node)
        routes = filteredRoutes[:]
        costs = filteredCosts[:]

    return (sol, solCost)



In [ ]:
import gurobipy as gp
import numpy as np


def computeMaxCost(d, a, b, n):
    # TODO: Use distance mat. to find an equivalent to infinity for subproblem
    return max([b[i] + d[i,j] - a[j] for i in range(n+2) for j in range(n+2)])


def setESPModelFO(model, x_vars, pi, n, d):
    rc = np.zeros((n+2,n+2))
    for i in range(n+2):
        for j in range(n+2):
            if (i==0) or (i==n+1):
                rc[i,j] = d[i,j]
            else:
                rc[i,j] = d[i,j] - pi[i-1]
    model.setObjective(gp.quicksum(x_vars[i,j]*rc[i,j] for i in range(n+2) \
                                                       for j in range(n+2)), \
                       gp.GRB.MINIMIZE)
    return


def createESPModel(d, pi, q, Q, a, b, n):
    M = computeMaxCost(d, a, b, n)
    model = gp.Model("ESPModel")
    x_vars = model.addVars(n+2, n+2, vtype=gp.GRB.BINARY, name="x")
    s_vars = model.addVars(n+2, vtype=gp.GRB.CONTINUOUS, name="s")

    setESPModelFO(model, x_vars, pi, n, d)

    # R0: capacity constraint
    model.addConstr(sum([q[i] * \
                    gp.quicksum(x_vars[i,j] for j in range(n+2)) \
                    for i in range(1,n+1)]) <= Q)
    # R1: depot start constraint
    model.addConstr(gp.quicksum(x_vars[0,j] for j in range(n+2)) == 1)
    # R2: depot finish constraint
    model.addConstr(gp.quicksum(x_vars[i,n+1] for i in range(n+2)) == 1)
    # R3-R53: flow costraints
    for h in range(1,n+1):
        model.addConstr(gp.quicksum(x_vars[i,h] for i in range(n+2)) - \
                        gp.quicksum(x_vars[h,j] for j in range(n+2)) == 0)

    # Time windows contraints
    for i in range(n+2):
        for j in range(n+2):
            if j!=i:
                model.addConstr(s_vars[i] + d[i,j] - M*(1-x_vars[i,j]) <= \
                                s_vars[j])

    # Service time constraints
    model.addConstrs(s_vars[i] >= a[i] for i in range(1,n+1))
    model.addConstrs(s_vars[i] <= b[i] for i in range(1,n+1))

    # Goodsense constraints:
    # Must not exist an arc that connects a customer with himself
    model.addConstr(gp.quicksum(x_vars[i,i] for i in range(n+2)) == 0)
    # No arc can enter in the first node
    model.addConstr(gp.quicksum(x_vars[i,0] for i in range(n+2)) == 0)
    # No arc can exit from the last node
    model.addConstr(gp.quicksum(x_vars[n+1,j] for j in range(n+2)) == 0)

    # model.write("ESPModel.lp")
    return model


## The main difference between RLCG_VRP and RLCG_CSP is the environment class

In [ ]:
from time import process_time
import os
from gurobipy import GRB
import gurobipy as gp
import numpy as np
import pandas as pd
import random
from copy import deepcopy


class VRP(object):
    def __init__(self, INSTANCE_NAME, n): ## input the name of that instance

        # each instance corresponds to self.state = self.env.reset() in learning_method in agent.py

        # state: curent graph (connection, current column nodes, current constraint node and their features)
        #### static info: problem defination, same info used for initialization this instance

        Kdim, Q, x, y, q, a, b = readData(INSTANCE_NAME, n)
        
        #### fixed info
        self.n = n
        self.Kdim = Kdim
        self.Q = Q
        self.x = x
        self.y = y
        self.q = q
        self.a = a
        self.b = b
        self.d = createDistanceMatrix(x, y)


        #### dynamic info for building RMP, PP


        ## update by addRoutetoMaster
        self.A = None ## like self.patterns as before
        self.c = None   # routes costs

        self.routes = None ## route list storing current routes
        

        #### dynamic info (info needed to for state + reward), get from CG iterations from solving current RMP and PP:  
        self.objVal_history = []
        self.total_steps = 0

        ## action with their reduced cost (stored as tuple) ([all the patterns],[(data for those routes)])
        self.available_action = ()

   

        self.count_convergence = 0

        '''  
        Info for column and constraint node features, stored using list,length will change
            column 
                      number of constraint participation
                      current solution value (if not in the basis, 0 -> int or not)
                      columnIsNew

                      column incompatibility degree --> check this

             constraint : shadow price
                          number of columns contributing to the constraint
        '''  
        ## for all the columns (size change)

        # self.RC = [] #### don't quite know how rc works here, so what i will do is that i will only have rc value for actions, all other rc is 0 ? 

        self.In_Cons_Num = []
        self.ColumnSol_Val = []
        self.ColumnIs_Basic = []
        
        ## for all the variable that are in the basis, count the number of times it's in basis, otherwise 0
        self.stay_in = []
        self.stay_out = []
        ## 1-> just left the basis in last iteration, 0 not just left
        self.just_left = []
        self.just_enter = []

        ## 1-> is action node, 0 -> .. useless as we can do this at get_aug_state
        # self.action_node = []



        ## for all the constraints (size fixed)
        self.Shadow_Price = None
        self.In_Cols_Num = []



    def generate_initial_patterns(self):

        impactSol = initializePathsWithImpact(self.d, self.n, self.a, self.b, self.q, self.Q)
        initial_routes = impactSol[:]
        self.routes = deepcopy(initial_routes)

        #print("Impact solution:", routes)

        # impactCost = sum([computeRouteCost(self.routes, self.d) for route in self.routes])


        A = np.zeros((self.n, len(self.routes)))
        c = np.zeros(len(self.routes))  
        addRoutesToMaster(self.routes, A, c, self.d)

        self.A = deepcopy(A)
        self.c = deepcopy(c)
        



    



    # get the constraint participation for each col node and col participation for each cons node
    ## use current patterns to count the non-zeros in the pattern matrix

    ### use A matrix -- read carefully how master and pp is built
    def update_col_con_number(self):
        A = deepcopy(self.A)
        self.In_Cons_Num = np.count_nonzero(A, axis=0)
        self.In_Cols_Num = np.count_nonzero(A, axis=1)
    



    def createMasterProblem(self):
        A = self.A
        costs = self.c
        n = self.n
        vehicleNumber = self.Kdim

        model = gp.Model("Master problem")
        model.Params.OutputFlag = 0
        y = model.addMVar(shape=A.shape[1], vtype=gp.GRB.CONTINUOUS, name="y")
        model.setObjective(costs @ y, gp.GRB.MINIMIZE)
        # Constraints
        model.addConstr(A @ y == np.ones(A.shape[0]))
        model.write("MasterModel.lp")
        model.Params.LogToConsole = 0

        return model


    # def solve_subproblem_return_actions(self, duals):
    # newRoutes = subProblem(n, q, d, a, b, rc, Q)

    def solve_subproblem_return_actions(self, rc, test=False):
       
        n = self.n
        q = self.q
        d = self.d
        readyt = self.a
        duedate = self.b
        Q = self.Q

        M = gp.GRB.INFINITY     # 1e+100
        # Time windows reduction
        a,b = reduceTimeWindows(n, d, readyt, duedate)
        # Reduce max capacity to boost algorithm
        if sum(q) < Q:
            Q = sum(q)
        T = max(b)

        # Init necessary data structure
        f = list()  # paths cost data struct
        p = list()  # paths predecessor data struct
        f_tk = list()     # cost of the best path that does not pass for
                          # predecessor (we'll call it alternative path)
        paths = []
        paths_tk = []
        for j in range(n+2):
            paths.append([])
            paths_tk.append([])
            for qt in range(Q-q[j]):
                paths[-1].append([])
                paths_tk[-1].append([])
                for tm in range(b[j]-a[j]):
                    paths[-1][-1].append([])
                    paths_tk[-1][-1].append([])
            mat = np.zeros((Q-q[j], b[j] - a[j]))
            p.append(mat - 1)
            f.append(mat + M)
            f_tk.append(mat + M)
        f[0][0,0] = 0
        f_tk[0][0,0] = 0
        L = set()   # Node to explore
        L.add(0)

        # Algorithm
        computation_counter = 0
        while L:
            if test: ## we will test on large instances, so set up some computation limits
                if computation_counter>=20:
                    computation_counter +=1
                    break
            else:
                computation_counter +=1 ## but for trainninng we want to train without such limits as instances are small
            

            i = L.pop()
            if i == n+1:
                continue

            # Explore all possible arcs (i,j)
            for j in range(1,n+2):
                if i == j:
                    continue
                for q_tk in range(q[i], Q-q[j]):
                    for t_tk in range(a[i], b[i]):
                        if p[i][q_tk-q[i], t_tk-a[i]] != j:
                            if f[i][q_tk-q[i], t_tk-a[i]] < M:
                                for t in range(max([a[j], int(t_tk+d[i,j])]),\
                                                    b[j]):
                                    if f[j][q_tk, t-a[j]]> \
                                      f[i][q_tk-q[i],t_tk-a[i]]+rc[i,j]:
                                        # if the current best path is suitable to
                                        # become the alternative path
                                        if p[j][q_tk, t-a[j]] != i \
                                          and p[j][q_tk, t-a[j]] != -1 \
                                          and f[j][q_tk, t-a[j]] < M \
                                          and f[j][q_tk,t-a[j]]<f_tk[j][q_tk,t-a[j]]:
                                            f_tk[j][q_tk,t-a[j]] = f[j][q_tk,t-a[j]]
                                            paths_tk[j][q_tk][t-a[j]] = \
                                                    paths[j][q_tk][t-a[j]][:]
                                        # update f
                                        f[j][q_tk,t-a[j]] = \
                                                f[i][q_tk-q[i],t_tk-a[i]] + rc[i,j]
                                        # update path that leads to node j
                                        paths[j][q_tk][t-a[j]] = \
                                                paths[i][q_tk-q[i]][t_tk-a[i]] + [j]
                                        # Update predecessor
                                        p[j][q_tk, t-a[j]] = i
                                        L.add(j)
                                    # if the path is suitable to be the alternative
                                    elif p[j][q_tk, t-a[j]] != i \
                                        and p[j][q_tk, t-a[j]] != -1 \
                                        and f_tk[j][q_tk, t-a[j]] > \
                                                f[i][q_tk-q[i],t_tk-a[i]]+rc[i,j]:
                                        f_tk[j][q_tk,t-a[j]] = \
                                                f[i][q_tk-q[i],t_tk-a[i]]+rc[i,j]
                                        paths_tk[j][q_tk][t-a[j]] = \
                                                paths[i][q_tk-q[i]][t_tk-a[i]]+[j]
                        else:       # if predecessor of i is j
                            if f_tk[i][q_tk-q[i], t_tk-a[i]] < M:
                                for t in range(max([a[j],int(t_tk+d[i,j])]), \
                                                    b[j]):
                                    if f[j][q_tk,t-a[j]] > \
                                            f_tk[i][q_tk-q[i],t_tk-a[i]]+rc[i,j]:
                                        # if the current best path is suitable to
                                        # become the alternative path
                                        if p[j][q_tk, t-a[j]] != i \
                                            and p[j][q_tk, t-a[j]] != -1 \
                                            and f[j][q_tk, t-a[j]] < M \
                                            and f[j][q_tk,t-a[j]] < \
                                                    f_tk[j][q_tk,t-a[j]]:
                                            f_tk[j][q_tk,t-a[j]] = f[j][q_tk,t-a[j]]
                                            paths_tk[j][q_tk][t-a[j]] = \
                                                    paths[j][q_tk][t-a[j]][:]
                                        # update f, path and bucket
                                        f[j][q_tk,t-a[j]] = \
                                            f_tk[i][q_tk-q[i],t_tk-a[i]] + rc[i,j]
                                        paths[j][q_tk][t-a[j]] = \
                                            paths_tk[i][q_tk-q[i]][t_tk-a[i]] + [j]
                                        p[j][q_tk,t-a[j]] = i
                                        L.add(j)
                                    # if the alternative path of i is suitable to
                                    # be the alternate of j
                                    elif p[j][q_tk, t-a[j]] != i \
                                        and p[j][q_tk, t-a[j]] != -1 \
                                        and f_tk[j][q_tk,t-a[j]] > \
                                                f_tk[i][q_tk-q[i],t_tk-a[i]]+rc[i,j]:
                                        f_tk[j][q_tk, t-a[j]] = \
                                            f_tk[i][q_tk-q[i],t_tk-a[i]]+rc[i,j]
                                        paths_tk[j][q_tk][t-a[j]] = \
                                            paths_tk[i][q_tk-q[i]][t_tk-a[i]] + [j]

        # Return all the routes with negative cost
        routes = list()
        rcosts = list()
        qBest, tBest = np.where(f[n+1] < -1e-9)

        for i in range(len(qBest)):
            newRoute = [0] + paths[n+1][qBest[i]][tBest[i]]
            if not newRoute in routes:
                routes.append(newRoute)
                rcosts.append(f[n+1][qBest[i]][tBest[i]])

        # print("New routes:", routes, flush=True)


        # print("reduced cost?", rcosts)


        costs = np.zeros(len(routes))
        for i in range(len(routes)):
            cost = d[routes[i][0],routes[i][1]]
            for j in range(1,len(routes[i])-1):
                cost += d[routes[i][j], routes[i][j+1]]
            costs[i] = cost

        costs = list(costs)
        # print("new routes",routes)


        return routes,rcosts,costs  ## return all avaliable actions (new routes generated, called routes here) and their rc

        

    def initialize(self,test_or_not=False):

        self.generate_initial_patterns()
        self.total_steps = 0
        routes = self.routes

        self.update_col_con_number()

        master_problem = self.createMasterProblem()
        master_problem.optimize()

        # # Compute reduced costs
        # constr = masterModel.getConstrs()
        # pi_i = [0.] + [const.pi for const in constr] + [0.]
        # for i in range(n+2):
        #     for j in range(n+2):
        #         rc[i,j] = d[i,j] - pi_i[i]

        # if not np.where(rc < -1e-9):
        #     break

        # self.RC = np.zeros(len(self.routes))


        self.ColumnSol_Val = np.asarray(master_problem.x)
        self.ColumnIs_Basic = np.asarray(master_problem.vbasis)+np.ones(len(routes))
        self.objVal_history.append(master_problem.objVal)


        dual_variables = [constraint.pi for constraint in master_problem.getConstrs()]
        self.Shadow_Price = dual_variables

        # Compute reduced costs
        constr = master_problem.getConstrs()
        pi_i = [0.] + [const.pi for const in constr] + [0.]

        rc = np.zeros((self.n+2,self.n+2))
        for i in range(self.n+2):
            for j in range(self.n+2):
                rc[i,j] = self.d[i,j] - pi_i[i]
        # print("reduced cost",rc)


        too_long = False
        time_before = time.time()
        columns_to_select,reduced_costs,route_costs = self.solve_subproblem_return_actions(rc,test_or_not)
        time_after = time.time()
        
        if not test:
            if time_after - time_before >=100.0: ## if the instance takes too long to solve during training, skip it
                too_long = True
        else:
            too_long = False ## test every instances

        self.available_action = (columns_to_select,[reduced_costs,route_costs])


        self.stay_in = list(np.zeros(len(routes)))
        self.stay_out = list(np.zeros(len(routes)))
        self.just_left = list(np.zeros(len(routes)))
        self.just_enter = list(np.zeros(len(routes)))

        reward = 0
        is_done = False
  
        return reward, is_done,too_long
        

    def step(self,action,test_or_not=False):
        self.total_steps +=1
        is_done = False

        ## historical info
        last_columns_to_select, columns_info = deepcopy(self.available_action) 
        last_basis= deepcopy(self.ColumnIs_Basic[:])
        last_basis = np.append(last_basis,0)


        self.routes.append(action)
        idx = 0
        for one_act in last_columns_to_select:
            if one_act == action:
                break
            idx+=1
        
        routes = deepcopy(self.routes)

        ## just append one cost
        self.c = np.append(self.c,columns_info[1][idx])

        ########################################
        # self.rc.appennd(columns_info[0][idx])
        ########################################

        



        
        # c = np.zeros(len(self.routes))  
        # addRoutesToMaster(self.routes, A, c, self.d)



        add_A = np.zeros((self.n, 1))

        for j in range(1,len(action)-1):
            add_A[action[j]-1] += 1

        # print(add_A)
        self.A = np.c_[self.A, add_A]

        self.update_col_con_number()

        master_problem = self.createMasterProblem()
        master_problem.optimize()

        dual_variables = [constraint.pi for constraint in master_problem.getConstrs()]
        self.Shadow_Price = dual_variables

        self.ColumnSol_Val = np.asarray(master_problem.x)
        self.ColumnIs_Basic = np.asarray(master_problem.vbasis)+np.ones(len(routes))

        #### you can either stay in the basis, leave the basis, or enter the basis
        difference = last_basis - self.ColumnIs_Basic 

        ### update the dynamic basis info based on difference
        self.just_left = list(np.zeros(len(difference)-1))
        self.just_enter = list(np.zeros(len(difference)-1))
        for i in range(len(difference)-1):
            if difference[i] == 1:
                
                self.just_left[i] = 1
                self.stay_in[i] = 0
            elif difference[i] == -1:
                
                self.just_enter[i] = 1
                self.stay_out[i] = 0
            elif difference[i] == 0:
                if last_basis[i] == 1:
                    self.stay_in[i]+=1
                else:
                    self.stay_out[i]+=1

        # append info for the new node; for just enter, look at column is basic
        self.just_left.append(0)
        self.stay_out.append(0)
        self.stay_in.append(0)

        if self.ColumnIs_Basic[-1] == 1:
            self.just_enter.append(1)
        else:
            self.just_enter.append(0)
   
        self.objVal_history.append(master_problem.objVal)



        rc = np.zeros((self.n+2,self.n+2))

        # Compute (rc, don't know what it is) used for building subproblem
        constr = master_problem.getConstrs()
        pi_i = [0.] + [const.pi for const in constr] + [0.]
        for i in range(self.n+2):
            for j in range(self.n+2):
                rc[i,j] = self.d[i,j] - pi_i[i]
        # print("reduced cost",rc)


        new_routes,action_rcosts,action_costs = self.solve_subproblem_return_actions(rc,test_or_not)
        if new_routes == []:
            is_done = True
            reward = 0.5*(self.objVal_history[-1] - self.objVal_history[0])

        else:
            self.available_action = (new_routes,[action_rcosts,action_costs])
#             reward = 100*(self.objVal_history[-2] - self.objVal_history[-1])/self.objVal_history[0] ## normalization term
        reward -=1



        return reward, is_done

  




 

        
        

        


## Now for the learning part

In [ ]:

import random
import numpy as np

from copy import deepcopy
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import matplotlib.pyplot as plt





class Agent(object):
  '''Base Class of Agent
  '''
  def __init__(self, initial_env=None, capacity = 10000):
      self.env = initial_env # the evironment would be one cutting stock object
      ## add the env and available action will be added in the learning_method 
      # self.A = self.env.available_action

      self.A = []
      self.experience = Experience(capacity = capacity)
      # S record the current super state for the agent
      # self.S = self.get_aug_state()   

      self.S = []


  ## get augmented state from the current environment
  def get_aug_state(self):

      
      # actions,reduced_costs = deepcopy(self.env.available_action)

      # print(self.env.A)

      actions,action_info = deepcopy(self.env.available_action)
      reduced_costs = action_info[0]
      
      total_added = len(actions)

      patterns = self.env.routes[:]

      is_action = np.asarray([0]*len(patterns))

      patterns.extend(actions)



      col_num = len(patterns)
      cons_num = self.env.n
      column_features = []
      cons_features = []
      edge_indices = [[],[]]




      # RC = self.env.RC[:]
      # RC = np.append(RC,reduced_costs)


      
      ###################################################
      MatA = deepcopy(self.env.A)
      cost_c = deepcopy(self.env.c)

      newMat = np.zeros((self.env.n, len(actions)))
      newCosts = np.zeros(len(actions))
      addRoutesToMaster(actions, newMat, newCosts, self.env.d) 

      # routes += newRoutes
      MatA = np.c_[MatA, newMat]

      In_Cons_Num = np.count_nonzero(MatA, axis=0)
      ###################################################


      ColumnSol_Val = self.env.ColumnSol_Val[:]
      ColumnSol_Val = np.append(ColumnSol_Val, np.zeros(total_added))
      cost_c = np.append(cost_c, newCosts)

      stay_in = self.env.stay_in[:]
      stay_in = np.append(stay_in,np.zeros(total_added))
      stay_out = self.env.stay_out[:]
      stay_out = np.append(stay_out,np.zeros(total_added))
      just_left = self.env.just_left[:]
      just_left = np.append(just_left,np.zeros(total_added))
      just_enter = self.env.just_enter[:]
      just_enter = np.append(just_enter,np.zeros(total_added))

      is_action = np.append(is_action,np.ones(total_added))
      


      #### constraint features, also augment all actions
      Shadow_Price = self.env.Shadow_Price[:]
      In_Cols_Num = np.count_nonzero(MatA, axis=1)

      # In_Cols_Num = self.env.In_Cols_Num[:]
      # for action in actions:
      #     non_zero = np.nonzero(action)
      #     for idx in non_zero:
      #         In_Cols_Num[idx]+=1
    
    

      Shadow_Price = np.asarray(Shadow_Price).reshape(-1, 1)
      In_Cons_Num = np.asarray(In_Cons_Num).reshape(-1, 1)
      In_Cols_Num = np.asarray(In_Cols_Num).reshape(-1, 1)
      ColumnSol_Val = np.asarray(ColumnSol_Val).reshape(-1, 1)
      cost_c = np.asarray(cost_c).reshape(-1, 1)
      stay_in = np.asarray(stay_in).reshape(-1, 1)
      stay_out = np.asarray(stay_out).reshape(-1, 1)


      from sklearn.preprocessing import MinMaxScaler
      # from sklearn.preprocessing import StandardScalar


      Scaler_SP = MinMaxScaler()
      Scaler_SP.fit(Shadow_Price)
      Shadow_Price = Scaler_SP.transform(Shadow_Price)
      Scaler_IConsN = MinMaxScaler()
      Scaler_IConsN.fit(In_Cons_Num)
      In_Cons_Num = Scaler_IConsN.transform(In_Cons_Num)
      Scaler_IColsN = MinMaxScaler()
      Scaler_IColsN.fit(In_Cols_Num)
      In_Cols_Num = Scaler_IColsN.transform(In_Cols_Num)
      Scaler_CSV = MinMaxScaler()
      Scaler_CSV.fit(ColumnSol_Val)
      ColumnSol_Val = Scaler_CSV.transform(ColumnSol_Val)
      Scaler_W = MinMaxScaler()
      Scaler_W.fit(cost_c)
      cost_c = Scaler_W.transform(cost_c)

      Scaler_si = MinMaxScaler()
      Scaler_si.fit(stay_in)
      stay_in = Scaler_si.transform(stay_in)

      Scaler_out = MinMaxScaler()
      Scaler_out.fit(stay_out)
      stay_out = Scaler_out.transform(stay_out)


      Shadow_Price = list(Shadow_Price.T[0])
      In_Cons_Num = list(In_Cons_Num.T[0])
      In_Cols_Num = list(In_Cols_Num.T[0])
      ColumnSol_Val = list(ColumnSol_Val.T[0])
      cost_c = list(cost_c.T[0])
      stay_in = list(stay_in.T[0])
      stay_out = list(stay_out.T[0])


      ### constraint nodes
      for j in range(cons_num):
          con_feat = []
          con_feat.append(Shadow_Price[j])
          con_feat.append(In_Cols_Num[j])
          cons_features.append(con_feat)

      
      ### normalize here for each information
      for i in range(col_num):
          col_feat = []
          col_feat.append(In_Cons_Num[i])
          col_feat.append(ColumnSol_Val[i])
          col_feat.append(cost_c[i])
          col_feat.append(stay_in[i])
          col_feat.append(stay_out[i])
          col_feat.append(just_left[i])
          col_feat.append(just_enter[i])
          col_feat.append(is_action[i])

          column_features.append(col_feat)
      
      ## get edges going
      for m in range(len(MatA[0])):
          for n in range(len(MatA)):
              if MatA[n][m]!=0:
                  # then mth column is connected to nth cons
                  edge_indices[0].append(m)
                  edge_indices[1].append(n)

      edge_indices = np.asarray(edge_indices)
      edge_indices[[0, 1]] = edge_indices[[1, 0]]


      cons_features=np.asarray(cons_features)
      column_features=np.asarray(column_features)

      ## need this total_added for reading the Q values, need actions to select onne pattern after read Q values
      aug_state, action_info = ((cons_features, edge_indices, column_features),(total_added,actions))
      return aug_state, action_info

  def policy(self):

      return random.choice(self.A)
      # return random.sample(self.A, k=1)[0]
  
  def perform_policy(self, s, Q = None, epsilon = 0.05):
      action = self.policy()
      return action


  def act(self, a0):
      ## get the current super state 
      s0_augmented, action_info_0 = self.S
      total_0 = deepcopy(action_info_0[0])
      # print(s0_augmented)

      ## step change the environnment, update all the information used for agent to construct state
      r, is_done = self.env.step(a0)

      s1_augmented, action_info_1 = self.get_aug_state()
      total_1 = action_info_1[0]
      trans = Transition(s0_augmented, a0, r, is_done, s1_augmented, action_info_0, total_0, total_1)
      total_reward = self.experience.push(trans)
      self.S = s1_augmented, action_info_1

      return s1_augmented, r, is_done, total_reward

  def learning_method(self,VRP_instance, gamma = 0.9, alpha = 1e-3, epsilon = 0.05):
      #self.state = self.env.reset()
      ## initialize an environment


      self.env = VRP_instance


      self.S = self.get_aug_state()

      ### initialize before calling
      # self.env.initialize()

      self.A = self.env.available_action[0]

      s0 = self.S
      a0 = self.perform_policy(s0, epsilon)
      time_in_episode, total_reward = 0, 0
      is_done = False
      while not is_done:
          # act also update self.S
          s1, r1, is_done, total_reward = self.act(a0)
          self.A = self.env.available_action[0]
          # if self.A == []:
          #     break;
          a1 = self.perform_policy(s1, epsilon)
          s0, a0 = s1, a1
          time_in_episode += 1

          # actions,reduced_costs = deepcopy(self.env.available_action)

      return time_in_episode, total_reward  
    

  def _decayed_epsilon(self,cur_episode: int, 
                            min_epsilon: float, 
                            max_epsilon: float, 
                            target_episode: int) -> float: 
      slope = (min_epsilon - max_epsilon) / (target_episode)
      intercept = max_epsilon
      return max(min_epsilon, slope * cur_episode + intercept)        
      
                      
  def learning(self,  epsilon = 0.05, decaying_epsilon = True, gamma = 0.9, 
                learning_rate = 3e-4, max_episode_num = 153, display = False, min_epsilon = 1e-2, min_epsilon_ratio = 0.8, model_index = 0):
      total_time,  episode_reward, num_episode = 0,0,0
      total_times, episode_rewards, num_episodes,history = [], [], [], []



      schedule = np.load("train_test/schedule.npy")

      for i in range(max_episode_num):
          if epsilon is None:
              epsilon = 1e-10
          elif decaying_epsilon:
              #epsilon = 1.0 / (1 + num_episode)
              epsilon = self._decayed_epsilon(cur_episode = num_episode+1,
                                              min_epsilon = min_epsilon,
                                              max_epsilon = epsilon,
                                              target_episode = int(max_episode_num * min_epsilon_ratio))
          

         
          #### read_file    

          # schedule = np.load("/content/drive/MyDrive/ICML_VRP/schedule.npy")
          n,problem_name = schedule[i]

          try:
              VRP_instance = VRP(problem_name,int(n))
          except:
              print(problem_name +" is not found, so skip")
              continue

          too_long = VRP_instance.initialize()[2]

          if too_long:
              print("Skip instance {} in episode {}".format(problem_name,i))
              continue



    

          
          time_in_episode, episode_reward = self.learning_method(VRP_instance, \
                gamma = gamma, learning_rate = learning_rate, epsilon = epsilon, display = display)
          # total_time += time_in_episode
          num_episode += 1

          total_times.append(time_in_episode)
          episode_rewards.append(episode_reward)
          num_episodes.append(num_episode)
          history.append(VRP_instance.objVal_history[-1])


          print("Episode: " + str(i) + " takes " + str(time_in_episode) +" steps with obj "+str(VRP_instance.objVal_history[-1]))



      




      model_save_name = 'Model.pt'
      path_model = 'Saved_results/{model_save_name}'
      self.target_Q.save_state(path_model)

    
 
      return  total_times, episode_rewards, num_episodes


  def sample(self, batch_size = 32):
      return self.experience.sample(batch_size)

  @property
  def total_trans(self):
      return self.experience.total_trans
  
  def last_episode_detail(self):
      self.experience.last_episode.print_detail()



In [ ]:

import numpy as np
from copy import deepcopy
import random


class DQNAgent(Agent):
    '''
    '''
    def __init__(self, env,
                       capacity,
                       hidden_dim,
                       batch_size,
                       epochs,
                       embedding_size,
                       cons_num_features,
                       vars_num_features,
                       learning_rate,
                       seed_):

        super(DQNAgent, self).__init__(env, capacity)
        self.embedding_size = embedding_size 
        self.hidden_dim = hidden_dim
        self.cons_num_features = cons_num_features
        self.vars_num_features = vars_num_features
        self.lr = learning_rate
        self.seed = seed_

        self.behavior_Q = BipartiteGNN(embedding_size = self.embedding_size, cons_num_features = self.cons_num_features, 
        vars_num_features = self.vars_num_features, learning_rate = self.lr, seed = self.seed)

        self.target_Q = BipartiteGNN(embedding_size = self.embedding_size, cons_num_features = self.cons_num_features, 
        vars_num_features = self.vars_num_features, learning_rate = self.lr, seed = self.seed)
        self._update_target_Q()
        
        self.batch_size = batch_size 
        self.epochs = epochs



        
    def _update_target_Q(self):
       
        '''
        '''
        self.target_Q.set_weights(deepcopy(self.behavior_Q.variables))
        # self.target_Q = self.behavior_Q.clone()
        
    
    ## s is the super s0, A is the list containing all actions
    def policy(self, action_info, s, epsilon = None):

        total_added, Actions = action_info
        Q_s = self.behavior_Q(s)
        Q_s_for_action = Q_s[-total_added::]

        rand_value = np.random.random()
        if epsilon is not None and rand_value < epsilon:
            return random.choice(list(Actions))
        else:
            idx = int(np.argmax(Q_s_for_action))
            return Actions[idx]


    ## s is the super s0, A is the list containing all actions
    ### need action info 0 and total 1 (total_1 to get max Q_1, action_info_0 to get update index)
    def get_max(self,  total_1, s):
        Q_s = self.target_Q.call(s)
        Q_s_for_action = Q_s[-total_1::]
        return np.max(Q_s_for_action)

    ## this method is used to get target in _learn_from_memory function
    ## select the max number from last few items from Q_matrix for each row, based on last_index_list
    
    def _learn_from_memory(self, gamma, learning_rate):

        ## trans_pieces is a list of transitions
        trans_pieces = self.sample(self.batch_size)  # Get transition data
        states_0 = np.vstack([x.s0 for x in trans_pieces]) # as s0 is a list, so vstack
        actions_0 = np.array([x.a0 for x in trans_pieces]) 
        reward_1 = np.array([x.reward for x in trans_pieces])
        is_dones = np.array([x.is_done for x in trans_pieces])
        states_1 = np.vstack([x.s1 for x in trans_pieces])
        action_info = np.vstack([x.action_info_0 for x in trans_pieces])
        totals_0 = np.vstack([x.total_0 for x in trans_pieces])
        totals_1 = np.vstack([x.total_1 for x in trans_pieces])

        y_batch = []
        for i in range(len(states_0)):
          
            ### get the index of action that is taken at s0
            acts_0 = action_info[i][1]
            act_0 = list(actions_0[i])

            idx = 0
            for act in acts_0:
                if act==act_0:
                    break
                idx+=1

            y = self.target_Q.call(states_0[i]).numpy()
            #### set the non action terms to be 0 
            y[0:-totals_0[i][0]] = 0

            if is_dones[i]:
                Q_target = reward_1[i]
            else:
                ### the number of actions for state 1 is used to get Q_target
                Q_max = self.get_max(totals_1[i][0], states_1[i])
                Q_target = reward_1[i] + gamma * Q_max

            y[-totals_0[i][0]+idx] = Q_target
            
            y_batch.append(np.asarray(y))

        y_batch= np.asarray(y_batch)
        X_batch = states_0
        
        loss = self.behavior_Q.train_or_test(X_batch, y_batch, totals_0, actions_0, action_info, True)
        # print("The loss is,", loss)
        self._update_target_Q()
      
        return loss

    #### the learning code
    def learning_method(self, instance, gamma, learning_rate, epsilon, 
                        display):

        epochs = self.epochs
        ###########
        self.env = instance
        self.S = self.get_aug_state()
        
        time_in_episode, total_reward = 0, 0
        is_done = False
        loss = 0
        while not is_done:
            s0_aug = self.S[0]
            action_info = self.S[1]
            ### a0 is selected based on behavior_Q
            # print(action_info[1])
            if len(action_info[1]) == 0:
                ## if no available actions,end this episode
                break;
            
            a0 = self.policy(action_info, s0_aug, epsilon)
            print("Added route is {} out of {}".format(a0,len(action_info[1])))

            s1_augmented, r, is_done, total_reward = self.act(a0)


            if self.total_trans > self.batch_size:
                for e in range(epochs):
                    loss += self._learn_from_memory(gamma, learning_rate)
                # loss/=epochs
            # s0 = s1
            time_in_episode += 1

        loss /= (time_in_episode*epochs)
        print("The loss is,",loss)
        if display:
            print("epsilon:{:3.2f},loss:{:3.2f},{}".format(epsilon,loss,self.experience.last_episode))
        return time_in_episode, total_reward  


In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as K
import os
import importlib


class BipartiteGNN(K.Model):

    '''
    Initialization of the different modules and attributes
    Attributes : 
    - embedding_size : Embedding size for the intermediate layers of the neural networks
    - cons_num_features : Number of constraint features, the constraints data matrix expected has the shape (None,cons_num_features)
    - vars_num_features : Number of variable features, the variables data matrix expected has the shape (None,vars_num_features)
    - learning_rate : Optimizer learning rate
    - activation : Activation function used in the neurons
    - initializer : Weights initializer
    '''
    def __init__(self, embedding_size = 32, cons_num_features = 2, 
        vars_num_features = 9, learning_rate = 1e-3, seed = 0,
        activation = K.activations.relu, initializer = K.initializers.Orthogonal):
        self.seed_value = seed
        tf.random.set_seed(self.seed_value)
        super(BipartiteGNN, self).__init__()


        self.embedding_size = embedding_size
        self.cons_num_features = cons_num_features
        self.vars_num_features = vars_num_features
        self.learning_rate = learning_rate
        self.activation = activation

        self.initializer = initializer(seed=self.seed_value)
        self.optimizer = tf.optimizers.Adam(learning_rate=self.learning_rate) 

        # constraints embedding layer
        self.cons_embedding = K.Sequential([
            K.layers.Dense(units=self.embedding_size, activation=self.activation, kernel_initializer=self.initializer),
        ])

        # variables/columns embedding layer
        self.var_embedding = K.Sequential([
            K.layers.Dense(units=self.embedding_size, activation=self.activation, kernel_initializer=self.initializer),
        ])

        # NN responsible for the intermediate updates
        self.join_features_NN = K.Sequential([
            K.layers.Dense(units=self.embedding_size, activation=self.activation, kernel_initializer=self.initializer),
            K.layers.Dense(units=self.embedding_size, activation=self.activation, kernel_initializer=self.initializer)
        ])

        # Representations updater for the constraints, called after the agregation
        self.cons_representation_NN = K.Sequential([
            K.layers.Dense(units=self.embedding_size, activation=self.activation, kernel_initializer=self.initializer),  
        ])
        # Representations updater for the variables/columns, called after the agregation
        self.vars_representation_NN = K.Sequential([
            K.layers.Dense(units=self.embedding_size, activation=self.activation, kernel_initializer=self.initializer),  
        ])

        # NN for final output, i.e., one unit logit output
        self.output_module = K.Sequential([
            K.layers.Dense(units=self.embedding_size, activation=self.activation, kernel_initializer=self.initializer),
            K.layers.Dense(units=self.embedding_size, activation=self.activation, kernel_initializer=self.initializer),
            K.layers.Dense(units=1, activation=None, kernel_initializer=self.initializer)
        ])

        # Build of the input shapes of all the NNs
        self.build()

        # Order set for loading/saving the model
        self.variables_topological_order = [v.name for v in self.variables]



    '''
    Build function, sets the input shapes. Called during initialization
    '''
    def build(self):
        self.cons_embedding.build([None, self.cons_num_features])
        self.var_embedding.build([None, self.vars_num_features])
        self.join_features_NN.build([None, self.embedding_size*2])
        self.cons_representation_NN.build([None, self.embedding_size*2])
        self.vars_representation_NN.build([None, self.embedding_size*2])
        self.output_module.build([None, self.embedding_size])
        self.built = True

    '''
    Main function taking as an input a tuple containing the three matrices :
    - cons_features : Matrix of constraints features, shape : (None, cons_num_features)
    - edge_indices : Edge indices linking constraints<->variables, shape : (2, None)
    - vars_features : Matrix of variables features, shape : (None, vars_num_features)

    Output : logit vector for the variables nodes, shape (None,1)
    '''
    def call(self, inputs):
        # print("The code is running", inputs[0].shape)

        cons_features, edge_indices, vars_features = inputs
        # print("#######################")
        # print(cons_features)
        # print(edge_indices)
        # print(vars_features)
        # print("#######################")
        # Nodes embedding, constraints and variables
        cons_features = self.cons_embedding(cons_features)
        vars_features = self.var_embedding(vars_features)

        # ==== First Pass : Variables -> Constraints ====
        # compute joint representations
        joint_features = self.join_features_NN(
                tf.concat([
                    tf.gather(
                        cons_features,
                        axis=0,
                        indices=edge_indices[0])
                    ,
                    tf.gather(
                        vars_features,
                        axis=0,
                        indices=edge_indices[1])
                    ### change this number to edge weights (patterns)
                ],1)
        )

        # Aggregation step
        output_cons = tf.scatter_nd(
            updates=joint_features,
            indices=tf.expand_dims(edge_indices[0], axis=1),
            shape=[cons_features.shape[0], self.embedding_size]
        )
        # Constraints representations update
        output_cons = self.cons_representation_NN(tf.concat([output_cons,cons_features],1))



        # ==== Second Pass : Constraints -> Variables ====
        # compute joint representations
        joint_features = self.join_features_NN(
                tf.concat([
                    tf.gather(
                        output_cons,
                        axis=0,
                        indices=edge_indices[0])
                    ,
                    tf.gather(
                        vars_features,
                        axis=0,
                        indices=edge_indices[1])
                ],1)
        )

        # Aggregation step
        output_vars = tf.scatter_nd(
            updates=joint_features,
            indices=tf.expand_dims(edge_indices[1], axis=1),
            shape=[vars_features.shape[0], self.embedding_size]
        )
        # Variables representations update
        output_vars = self.vars_representation_NN(tf.concat([output_vars,vars_features],1))

        # ==== Final output from the variables representations (constraint nodes are ignored)
        output = self.output_module(output_vars)
        return output

    '''
    Save model and current weights to a given path
    '''
    def save_state(self, path):
        import pickle
        with open(path, 'wb') as f:
            for v_name in self.variables_topological_order:
                v = [v for v in self.variables if v.name == v_name][0]
                pickle.dump(v.numpy(), f)

    '''
    Load an existing model from a given path
    '''
    def restore_state(self, path):
        import pickle
        with open(path, 'rb') as f:
            for v_name in self.variables_topological_order:
                v = [v for v in self.variables if v.name == v_name][0]
                v.assign(pickle.load(f))

    def get_config(self):
        config = {}
        for i in range(len(self.variables_topological_order)):
            config[self.variables_topological_order[i]] = self.variables[i]
        return config

    @staticmethod
    def from_config(model):
        for v_name in self.variables_topological_order:
            v = [v for v in model.variables if v.name == v_name][0]
            v.assign(config[v])
        return model
       

    '''
    Training/Test function
    Input: 
    - data : a batch of data, type : tf.data.Dataset
    - train: boolean, True if function called for training (i.e., compute gradients and update weights),
                False if called for test
    Output:
    tuple(Loss, Accuracy, Recall, TNR) : Metrics
    '''
    def train_or_test(self, data, labels, totals_0, actions_0,action_info, train=False):
        mean_loss = 0
  
        batches_counter = 0

        ###########################################################
        ### how does this data(a batch) relates to transition data?
        ###########################################################
        for batch in data:
            cons_features, edge_indices, vars_features = batch
            input_tuple = (cons_features, edge_indices, vars_features)

            total_0 = totals_0[batches_counter]

            label = labels[batches_counter]



            # When called train=True, compute gradient and update weights
            if train:
                with tf.GradientTape() as tape:
                    # Get logits from the bipartite GNN model

                    ########
                    ######### may need to change to self.call?
                    logits = self.call(input_tuple)
                    # print(total_0)
                    label[0:-total_0[0]] =  logits[0:-total_0[0]] ## do not count the loss from the nodes already in the basis
 
                    loss = tf.keras.metrics.mean_squared_error(label,logits) ## should not be mean_squared_error as it's then scaled down by number of nodes
                    # loss = (loss * label.shape[0]) / total_0[0]  ## this is a quick fix, as there are far less action nodes compared to 
                    loss = (loss * label.shape[0])
                # Compute gradient and update weights
                grads = tape.gradient(target=loss, sources=self.variables)
                self.optimizer.apply_gradients(zip(grads, self.variables))
            # If no optimizer instance set, no training is performed, give outputs and metrics only
            else:
                logits = self.call(input_tuple)
                loss = tf.keras.metrics.mean_squared_error(label,logits)

            loss = tf.reduce_mean(loss)

            # Batch loss, accuracy, confusion matrix
            mean_loss += loss
            batches_counter += 1 
            # confusion_mat += confusion_matrix(labels, prediction)

        # Batch average loss
        mean_loss /= batches_counter
        return mean_loss




In [ ]:
# Model 3:  (300, 0.05, 0.9, 0.001)

class PARAMETERS(object):
    def __init__(self):
        self.seed = 5


        ## parameters about neural network
        self.lr = 1e-3  ##
        self.batch_size = 16
        self.hidden_dim = 32
        self.epochs = 5
        self.embedding_size = 32
        self.cons_num_features = 2
        self.vars_num_features = 8



        ## parameters of RL algorithm
        self.gamma = 0.99 ##
        self.epsilon = 0.2
        self.min_epsilon = 0.2
        self.min_epsilon_ratio = 0.99
        self.decaying_epsilon = False
        self.step_penalty = 1
        self.alpha_obj_weight = 5 ##
        self.action_pool_size = 10 
        self.max_episode_num = 447
        self.capacity = 20000 


        self.model_index = 3 #####



## training

In [ ]:

import matplotlib
matplotlib.use('Agg')
import os.path
import numpy as np
import time
import random
import numpy as np


Parameters = PARAMETERS()

random.seed(Parameters.seed)
np.random.seed(Parameters.seed)

epsilon_ = Parameters.epsilon
decaying_epsilon_ = Parameters.decaying_epsilon
gamma_ = Parameters.gamma
alpha_ = Parameters.alpha_obj_weight
max_episode_num_ = Parameters.max_episode_num
min_epsilon_ = Parameters.min_epsilon
min_epsilon_ratio_ = Parameters.min_epsilon_ratio
capacity_ =  Parameters.capacity
hidden_dim_ = Parameters.hidden_dim
batch_size_ = Parameters.batch_size
epochs_ = Parameters.epochs
embedding_size_ = Parameters.embedding_size
cons_num_features_ = Parameters.cons_num_features
vars_num_features_ = Parameters.vars_num_features
learning_rate_ = Parameters.lr
model_index_ = Parameters.model_index
seed_fix = Parameters.seed

display_ = False


### training and saving the data for plotting and model weights (weights and data are saved inside .learning)


DQN = DQNAgent(env = None, capacity = capacity_, hidden_dim = hidden_dim_, batch_size = batch_size_, epochs = epochs_, embedding_size = embedding_size_, 
			   cons_num_features = cons_num_features_, vars_num_features = vars_num_features_, learning_rate = learning_rate_, seed_ = seed_fix)



time_start = time.time()
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)     
total_times, episode_rewards, num_episodes =  DQN.learning(epsilon = epsilon_, decaying_epsilon = decaying_epsilon_, gamma = gamma_, 
                learning_rate = learning_rate_, max_episode_num = max_episode_num_, display = display_, min_epsilon = min_epsilon_, min_epsilon_ratio = min_epsilon_ratio_, model_index = model_index_)    

time_ends = time.time()

### training will take quite some time as the pricing problem in this case (VRPTW) takes more time than pricing problem in CSP


## testing after training

In [ ]:

Parameters = PARAMETERS()
def follow_policy(DQN,action_info, s):
    '''DQN selects an action 
    '''
    total_added, Actions = action_info
    Q_s = DQN.target_Q(s)
    Q_s_for_action = Q_s[-total_added::]
    # rand_value = np.random.random()
    idx = int(np.argmax(Q_s_for_action))
    return Actions[idx]

import time
def general_compare(DQN_test):

    Greedy=[]
    # Expert = []
    RL=[]

    True_obj = [] 

    print("#####################")
    print("Starts testing for model")
    print()
    
    model_save_name ='Model.pt'
    schedule_test = np.load('train_test/test.npy',allow_pickle=True)
    path_model = 'Saved_results/{model_save_name}'
 
    DQN.target_Q.restore_state(path_model)
    DQN.behavior_Q.restore_state(path_model)


    for k in range(len(schedule_test)):
        print("#### {} instance out of {}".format(k,len(schedule_test)))
        
        n,problem_name = schedule_test[k]

        ################################## Greedy
        try:
            VRP_instance = VRP(problem_name,int(n))
        except:
            print("Skip as not found")
            continue
        

        too_long = VRP_instance.initialize()[2]
        if too_long:
            print("skip instance "+problem_name)
            continue
        print("starts greedy")

        t1 = time.time()
        is_done = False
        while True:
            if is_done:
                break

            action = VRP_instance.available_action[0][-1]
            reward, is_done = VRP_instance.step(action)
            print("Added route is {} out of {}".format(action,len(VRP_instance.available_action[0])))
        t2 = time.time()

        history_opt_g = VRP_instance.objVal_history


        obj_greedy = history_opt_g[-1]
        steps_g = len(history_opt_g)
        print("Greedy takes {} steps to reach obj {} with time {}".format(steps_g,obj_greedy,t2-t1))
        # print("starts RL")
        
        ###################################  RL
        try:
            VRP_instance2 = VRP(problem_name,int(n))
        except:
            print("Skip as not found")
            continue
            
        too_long = VRP_instance2.initialize()[2]
        t3 = time.time()
        DQN.env = VRP_instance2
        DQN.S = DQN.get_aug_state()
        
        is_done = False
        while True:
            if is_done:
                break

            action_info = DQN.S[1]
            s = DQN.S[0]
            action = follow_policy(DQN,action_info,s)
            reward, is_done = VRP_instance2.step(action)
            DQN.S = DQN.get_aug_state()

            print("Added route is {} out of {}".format(action,len(action_info[1])))

        t4 = time.time()

        history_opt_rl = VRP_instance2.objVal_history

        obj_RL = history_opt_rl[-1]
        steps_RL = len(history_opt_rl)

        print("RL takes {} steps to reach obj {} with time {}".format(steps_RL,obj_RL,t4-t3))
        
        #### (full history, total number of steps, times, obj value)
        Greedy.append((history_opt_g,len(history_opt_g),t2-t1,obj_greedy))
        RL.append((history_opt_rl,len(history_opt_rl),t4-t3,obj_RL))

        # Expert.append((history_opt_expert,len(history_opt_expert),time4-time3,obj_expert))
        print()
        print("{} steps out of {}".format(k,len(schedule_test)))
        print("#########")
        
        
        ## save results after every 5 instances
        if k%5==0:
            path = 'Saved_results/data/'
            np.save(path+'test_results_ '+str(k),(Greedy,RL))

    complete_data = (Greedy,RL)
    path = 'Saved_results/data/'
    np.save(path+'FULL_TEST_RESULTS_LIST',complete_data)

    #### return three lists of tuple: (running time, steps)

    return complete_data



    




## average result of validation

In [4]:
import numpy as np
import pandas as pd

In [5]:
DATA = np.load('Saved_results/data/FULL_TEST_RESULTS_LIST.npy',allow_pickle=True)
NAME = np.load('train_test/test.npy',allow_pickle=True)

In [7]:
def mean_std(vec):
  return [vec.mean().round(1), vec.std().round(1)]

# def stat(arr):
#   return [mean_std(arr[i, :]) for i in range(3)]

def stat(arr):
  return [mean_std(arr[i, :]) for i in range(2)]


def result_table(data):
  iternum = data[:, :, 1].astype('float64')
  time = data[:, :, 2].astype('float64')
  objval = data[:, :, 3].astype('float64')
  iter_tab = pd.DataFrame(data = np.asarray(stat(iternum)), columns = ["Mean", "Std"])
  iter_tab.index = ["Greedy", "RL"]
  time_tab = pd.DataFrame(data = np.asarray(stat(time)), columns = ["Mean", "Std"])
  time_tab.index = ["Greedy", "RL"]
  objval_tab = pd.DataFrame(data = np.asarray(stat(objval)), columns = ["Mean", "Std"])
  objval_tab.index = ["Greedy", "RL"]
  return iter_tab, time_tab, objval_tab

In [8]:
res_tabs = result_table(DATA)

In [9]:
res_tabs[0]

,Mean,Std
Greedy,135.5,101.9
RL,76.9,38.0


In [10]:
res_tabs[1]

,Mean,Std
Greedy,3635.1,4837.9
RL,1917.9,1963.3


In [11]:
res_tabs[2]

,Mean,Std
Greedy,466.8,142.1
RL,466.8,142.1
